## Preparation

In [2]:
!pip install cx_Oracle

     -------------------------------------- 213.1/213.1 kB 2.2 MB/s eta 0:00:00


In [19]:
# importing the necessary libraries

import cx_Oracle
import pandas as pd
import re

In [21]:
# setting up a connection to database

dsn = "localhost/xepdb1"
username = "marta"    #local user
password = "marta"

conn = cx_Oracle.connect(username, password, dsn)
print(conn.version)

21.3.0.0.0


In [22]:
# creating a cursor

cursor = conn.cursor()

In [70]:
# checking which tables the local user has access to

query = "SELECT * FROM user_tables"  # no semicolon
cursor.execute(query)
results = cursor.fetchall()

print(pd.DataFrame(results).iloc[:, :1].values)

[['CUSTOMERS']
 ['VISITS']
 ['WASHES']
 ['MACHINES']
 ['FABRICS']
 ['PROGRAMS']
 ['SERVICES']
 ['COMPLAINTS']]


## Automation of reading queries from a file

In [350]:
# opening SQL file

file = open('queries.sql', 'r')
queries = file.read()
file.close()

In [351]:
# splitting the file into list of individual tasks

queries = re.split(";\n\n--\d\. ", queries)

In [352]:
# splitting according to the element type (task description/individual queries) 

queries_split = []
for q in queries:
    queries_split.append(q.split('\n\n'))

In [353]:
len(queries_split[8])

2

In [370]:
a = "sjiji"
b = ["hfhsif", "dhdsf", "dkajdk"]

q = []
q.append([a]+b)
q=[a]+b
q

['sjiji', 'hfhsif', 'dhdsf', 'dkajdk']

In [432]:
queries2 = []

for q in queries_split:
    task = re.sub("^.*\d\. ", "", q[0])  # task description taking into consideration previous side effect of split
    
    # solution queries (partial and individual)
    elements = []
    for element in q[1:]:
        element = re.sub("--.*\n", "\n", element)     # removing all comments from partial and individual queries
        element = re.sub("\n", " ", element)          # removing new line characters
        element = re.sub("\s+", " ", element)         # removing multiple spaces
        element = re.sub(";$", "", element)           # removing the semicolon at the end
        element = re.sub("^\s", "", element)          # removing white spaces in the beginning
        elements.append(element)
        
#     print('\n\n')
    queries2.append([task] + elements)

In [436]:
# steps taken shown in sample record

for q in queries_split[2:3]:
    print("Inicial task description:\n", repr(q[0]), "\n")
    task = re.sub("^.*\d\. ", "", q[0])  # task description taking into consideration previous side effect of split
    print("Final task description:\n", repr(task), "\n")
    print("-"*30)
    
    # solution queries (partial and individual)
    for element in q[1:]:
        print("\nInicial query:\n", repr(element), "\n")
        element = re.sub("--.*\n", "\n", element)     # removing all comments from partial and individual queries
        element = re.sub("\n", " ", element)          # removing new line characters
        element = re.sub("\s+", " ", element)         # removing multiple spaces
        element = re.sub(";$", "", element)           # removing the semicolon at the end
        element = re.sub("^\s", "", element)          # removing white spaces in the beginning
        print("Final query:\n", repr(element), "\n")
        print("-"*30)

Inicial task description:
 'Display the category of complaints that received the most complaints in 2021, along with the number of those complaints.' 

Final task description:
 'Display the category of complaints that received the most complaints in 2021, along with the number of those complaints.' 

------------------------------

Inicial query:
 "CREATE OR REPLACE VIEW catcomplaintno AS                        --view with aggregated data\n    SELECT\n        complaint_category,\n        COUNT(complaint_id) complaints_no\n    FROM\n        complaints\n    WHERE\n        TO_CHAR(submission_date, 'yyyy') = '2021'\n    GROUP BY\n        complaint_category;" 

Final query:
 "CREATE OR REPLACE VIEW catcomplaintno AS SELECT complaint_category, COUNT(complaint_id) complaints_no FROM complaints WHERE TO_CHAR(submission_date, 'yyyy') = '2021' GROUP BY complaint_category" 

------------------------------

Inicial query:
 'SELECT\n    *\nFROM\n    catcomplaintno;' 

Final query:
 'SELECT * FROM c

## Querying the database

**Note:** some tasks have more than one solution. For this reason, messages have been added indicating where the query was called and when the view was created.

In [414]:
for i in range(0, len(queries2)):
    
    # display the task description and enumerate
    print("\n",i+1,".", queries2[i][0], "\n")
    
    # execute the queries
    for q in queries2[i][1:]:
        
        # SELECT statements and displaying the result (with column names and no index)
        if re.match("^SELECT", q):
            print("Query executed.")
            ex = cursor.execute(q)
            results = cursor.fetchall()
            display(pd.DataFrame(results, columns=[cursor.description[i][0] for i in range(len(cursor.description))]).style.hide())
        
        # CREATE OR REPLACE VIEW statements
        else:
            print("A view created.")
            ex = cursor.execute(q)
    print("\n")


 1 . Display a list of individual visits on which at least one free drying service was performed. 

Query executed.


Visit ID,Number of drying services
7,2
3,2
9,2
1,2
5,1
4,1





 2 . Display the name of the highest temperature program for individual washing machines. 

Query executed.


MACHINE_NO,PROGRAM_NO,TEMPERATURE
1,1,60
1,4,60
2,6,90
2,7,90
3,12,60
3,13,60
4,17,50
4,18,50
4,19,50
5,6,90





 3 . Display the category of complaints that received the most complaints in 2021, along with the number of those complaints. 

A view created.
Query executed.


...

Rating,Number of additional powder bags,Number of free dryings
very good,6,5
good,4,4
average,1,0
bad,0,1


Query executed.


Rating,Free additions used
very good,11
good,8
bad,1
average,1





 9 . Display how many days after the visits complaints were received. 

Query executed.


DATE_TIME,SUBMISSION_DATE,Days difference
2021-08-10 14:10:10,2021-08-15 00:00:00,5
2021-09-03 16:40:10,2021-09-04 00:00:00,1
2021-09-03 16:40:10,2021-09-04 00:00:00,1
2021-09-05 09:32:00,2021-09-07 00:00:00,2
2021-09-05 09:32:00,2021-09-07 00:00:00,2
2021-09-05 10:46:08,2021-09-08 00:00:00,3
2021-09-05 10:46:08,2021-09-08 00:00:00,3


In [438]:
# disabling the cursor and closing the connection

cursor.close()
conn.close()